<a href="https://colab.research.google.com/github/aj1365/DeepCNN_Polsar/blob/main/DeepCNNwithHaarTrnasformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization, MaxPool2D, MaxPooling1D,Add, ConvLSTM2D, LSTM, Conv1D
from keras.layers import Dropout, Input
from tensorflow.keras.models import Model
#from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras import backend as Kb
from keras.layers import Lambda
from keras.layers import Activation
from keras.layers.merge import add, concatenate
from keras.layers import AveragePooling2D
#from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import plot_model
#from keras.utils import plot_model
import tensorflow 
 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
 
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import PCA
from operator import truediv
 
from plotly.offline import init_notebook_mode
 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
#!pip install spectral
import spectral

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
def loadData(name):
    
    data_path = os.path.join(os.getcwd(),'E:/PolSAR/')
   
    if name == 'Flevoland':
        
        data = sio.loadmat(os.path.join(data_path, 'Flevoland_T3RF.mat'))['T3RF']
        labels = sio.loadmat(os.path.join(data_path, 'Flevoland_15cls.mat'))['label']
  
    if name == 'SanFrancisco':
        
        data = sio.loadmat(os.path.join(data_path, 'SanFrancisco_T3RF.mat'))['T3RF']
        labels = sio.loadmat(os.path.join(data_path, 'SanFrancisco_gt.mat'))['SanFrancisco_gt']

    
    return data, labels

In [ ]:
## GLOBAL VARIABLES
dataset = 'Flevoland'
test_ratio = 0.90
windowSize = 12

In [ ]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [ ]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [ ]:
def createImageCubes(X, y, windowSize=8, removeZeroLabels = True):
    margin = int((windowSize) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin , c - margin:c + margin ]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [ ]:
X, Y = loadData(dataset)

#X=(X-np.min(X))/(np.max(X)-np.min(X))
X.shape, Y.shape

In [ ]:
K = X.shape[2]

In [ ]:
X2, Y2 = createImageCubes(X, Y, windowSize=12)

#X, y = createImageCubes(X, y, windowSize=windowSize,  removeZeroLabels = False)

#X.shape, y.shape
X.shape, Y.shape

In [ ]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X2, Y2, test_ratio)

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape



In [ ]:
output_units = 15
input_shape = (12, 12, 12)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [ ]:
def WaveletTransformAxisY(batch_img):
    odd_img  = batch_img[:,0::2]
    even_img = batch_img[:,1::2]
    L = (odd_img + even_img) / 2.0
    H = Kb.abs(odd_img - even_img)
    return L, H

def WaveletTransformAxisX(batch_img):
    # transpose + fliplr
    tmp_batch = Kb.permute_dimensions(batch_img, [0, 2, 1])[:,:,::-1]
    _dst_L, _dst_H = WaveletTransformAxisY(tmp_batch)
    # transpose + flipud
    dst_L = Kb.permute_dimensions(_dst_L, [0, 2, 1])[:,::-1,...]
    dst_H = Kb.permute_dimensions(_dst_H, [0, 2, 1])[:,::-1,...]
    return dst_L, dst_H

In [ ]:
def Wavelet(batch_image):
    # make channel first image
    batch_image = Kb.permute_dimensions(batch_image, [0, 3, 1, 2])
    r = batch_image[:,0]
    g = batch_image[:,1]
    b = batch_image[:,2]

    # level 1 decomposition
    wavelet_L, wavelet_H = WaveletTransformAxisY(r)
    r_wavelet_LL, r_wavelet_LH = WaveletTransformAxisX(wavelet_L)
    r_wavelet_HL, r_wavelet_HH = WaveletTransformAxisX(wavelet_H)

    wavelet_L, wavelet_H = WaveletTransformAxisY(g)
    g_wavelet_LL, g_wavelet_LH = WaveletTransformAxisX(wavelet_L)
    g_wavelet_HL, g_wavelet_HH = WaveletTransformAxisX(wavelet_H)

    wavelet_L, wavelet_H = WaveletTransformAxisY(b)
    b_wavelet_LL, b_wavelet_LH = WaveletTransformAxisX(wavelet_L)
    b_wavelet_HL, b_wavelet_HH = WaveletTransformAxisX(wavelet_H)

    wavelet_data = [r_wavelet_LL, r_wavelet_LH, r_wavelet_HL, r_wavelet_HH, 
                    g_wavelet_LL, g_wavelet_LH, g_wavelet_HL, g_wavelet_HH,
                    b_wavelet_LL, b_wavelet_LH, b_wavelet_HL, b_wavelet_HH]
    transform_batch = Kb.stack(wavelet_data, axis=1)

    # level 2 decomposition
    wavelet_L2, wavelet_H2 = WaveletTransformAxisY(r_wavelet_LL)
    r_wavelet_LL2, r_wavelet_LH2 = WaveletTransformAxisX(wavelet_L2)
    r_wavelet_HL2, r_wavelet_HH2 = WaveletTransformAxisX(wavelet_H2)

    wavelet_L2, wavelet_H2 = WaveletTransformAxisY(g_wavelet_LL)
    g_wavelet_LL2, g_wavelet_LH2 = WaveletTransformAxisX(wavelet_L2)
    g_wavelet_HL2, g_wavelet_HH2 = WaveletTransformAxisX(wavelet_H2)

    wavelet_L2, wavelet_H2 = WaveletTransformAxisY(b_wavelet_LL)
    b_wavelet_LL2, b_wavelet_LH2 = WaveletTransformAxisX(wavelet_L2)
    b_wavelet_HL2, b_wavelet_HH2 = WaveletTransformAxisX(wavelet_H2)


    wavelet_data_l2 = [r_wavelet_LL2, r_wavelet_LH2, r_wavelet_HL2, r_wavelet_HH2, 
                    g_wavelet_LL2, g_wavelet_LH2, g_wavelet_HL2, g_wavelet_HH2,
                    b_wavelet_LL2, b_wavelet_LH2, b_wavelet_HL2, b_wavelet_HH2]
    transform_batch_l2 = Kb.stack(wavelet_data_l2, axis=1)

    # level 3 decomposition
    wavelet_L3, wavelet_H3 = WaveletTransformAxisY(r_wavelet_LL2)
    r_wavelet_LL3, r_wavelet_LH3 = WaveletTransformAxisX(wavelet_L3)
    r_wavelet_HL3, r_wavelet_HH3 = WaveletTransformAxisX(wavelet_H3)

    wavelet_L3, wavelet_H3 = WaveletTransformAxisY(g_wavelet_LL2)
    g_wavelet_LL3, g_wavelet_LH3 = WaveletTransformAxisX(wavelet_L3)
    g_wavelet_HL3, g_wavelet_HH3 = WaveletTransformAxisX(wavelet_H3)

    wavelet_L3, wavelet_H3 = WaveletTransformAxisY(b_wavelet_LL2)
    b_wavelet_LL3, b_wavelet_LH3 = WaveletTransformAxisX(wavelet_L3)
    b_wavelet_HL3, b_wavelet_HH3 = WaveletTransformAxisX(wavelet_H3)

    wavelet_data_l3 = [r_wavelet_LL3, r_wavelet_LH3, r_wavelet_HL3, r_wavelet_HH3, 
                    g_wavelet_LL3, g_wavelet_LH3, g_wavelet_HL3, g_wavelet_HH3,
                    b_wavelet_LL3, b_wavelet_LH3, b_wavelet_HL3, b_wavelet_HH3]
    transform_batch_l3 = Kb.stack(wavelet_data_l3, axis=1)

    # level 4 decomposition
    wavelet_L4, wavelet_H4 = WaveletTransformAxisY(r_wavelet_LL3)
    r_wavelet_LL4, r_wavelet_LH4 = WaveletTransformAxisX(wavelet_L4)
    r_wavelet_HL4, r_wavelet_HH4 = WaveletTransformAxisX(wavelet_H4)

    wavelet_L4, wavelet_H4 = WaveletTransformAxisY(g_wavelet_LL3)
    g_wavelet_LL4, g_wavelet_LH4 = WaveletTransformAxisX(wavelet_L4)
    g_wavelet_HL4, g_wavelet_HH4 = WaveletTransformAxisX(wavelet_H4)

    wavelet_L3, wavelet_H3 = WaveletTransformAxisY(b_wavelet_LL3)
    b_wavelet_LL4, b_wavelet_LH4 = WaveletTransformAxisX(wavelet_L4)
    b_wavelet_HL4, b_wavelet_HH4 = WaveletTransformAxisX(wavelet_H4)


    wavelet_data_l4 = [r_wavelet_LL4, r_wavelet_LH4, r_wavelet_HL4, r_wavelet_HH4, 
                    g_wavelet_LL4, g_wavelet_LH4, g_wavelet_HL4, g_wavelet_HH4,
                    b_wavelet_LL4, b_wavelet_LH4, b_wavelet_HL4, b_wavelet_HH4]
    transform_batch_l4 = Kb.stack(wavelet_data_l4, axis=1)

    # print('shape before')
    # print(transform_batch.shape)
    # print(transform_batch_l2.shape)
    # print(transform_batch_l3.shape)
    # print(transform_batch_l4.shape)

    decom_level_1 = Kb.permute_dimensions(transform_batch, [0, 2, 3, 1])
    decom_level_2 = Kb.permute_dimensions(transform_batch_l2, [0, 2, 3, 1])
    decom_level_3 = Kb.permute_dimensions(transform_batch_l3, [0, 2, 3, 1])
    decom_level_4 = Kb.permute_dimensions(transform_batch_l4, [0, 2, 3, 1])
    
    # print('shape after')
    # print(decom_level_1.shape)
    # print(decom_level_2.shape)
    # print(decom_level_3.shape)
    # print(decom_level_4.shape)
    return [decom_level_1, decom_level_2]


def Wavelet_out_shape(input_shapes):
    # print('in to shape')
    return [tuple([None, 112, 112, 12]), tuple([None, 56, 56, 12]), 
            tuple([None, 28, 28, 12]), tuple([None, 14, 14, 12])]

In [ ]:
img_batch = Kb.zeros(shape=(12, 12, 12, 3), dtype='float32')
Wavelet(img_batch)

In [ ]:
def get_wavelet_cnn_model():
    
    input_shape =  12, 12, 12
    #imIn=Input(shape=input_shape)
 
    input_ = Input(input_shape, name='the_input')
    # wavelet = Lambda(Wavelet, name='wavelet')
    wavelet = Lambda(Wavelet, Wavelet_out_shape, name='wavelet')
    input_l1, input_l2= wavelet(input_)
    
###################################################################################
############################ First CNN
    
    conv_a1 = Conv2D(filters=96, kernel_size=(11,11), activation='relu',padding='same',  name='conv_a1')(input_)
    max_a1 = MaxPool2D(pool_size=(2,2))(conv_a1)
    
    conv_a2 = Conv2D(filters=256, kernel_size=(5,5), activation='relu',padding='same',  name='conv_a2')(max_a1)
    max_a3 = MaxPool2D(pool_size=(1,1))(conv_a2)
    
    conv_a3 = Conv2D(filters=384, kernel_size=(3,3), activation='relu',padding='same',  name='conv_a3')(max_a3)
    conv_a4 = Conv2D(filters=384, kernel_size=(3,3), activation='relu',padding='same',  name='conv_a4')(conv_a3)
    conv_a5 = Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding='same',  name='conv_a5')(conv_a4)
    
  
 ###########################################################
###################### Second CNN ALexNet
   
    conv_b1 = Conv2D(filters=32, kernel_size=(3, 3), padding='same', name='conv_b1')(input_l1)
    max_b_1 = MaxPool2D(pool_size=(1,1))(conv_b1)
    conv_b2 = Conv2D(filters=32, kernel_size=(3, 3), padding='same', name='conv_b2')(max_b_1)
    conv_b3 = Conv2D(filters=64, kernel_size=(3, 3), padding='same', name='conv_b3')(conv_b2)
    norm_b = BatchNormalization(name='norm_a')(conv_b3)
    relu_b = Activation('relu', name='relu_a')(norm_b)
    max_b_2 = MaxPool2D(pool_size=(1,1))(relu_b)
   
 #################################### 
    ############################### concate first  CNN to Second  CNN
    
    concate_level_1 = concatenate([conv_a5, max_b_2])
    conv_c2 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_2')(concate_level_1)
    norm_c2 = BatchNormalization(name='norm_2')(conv_c2)
    relu_c2 = Activation('relu', name='relu_2')(norm_c2)
     
    conv_c2_2 = Conv2D(256, kernel_size=(1, 1), strides=(2, 2), padding='same', name='conv_2_2')(relu_c2)
    norm_c2_2 = BatchNormalization(name='norm_2_2')(conv_c2_2)
    relu_c2_2 = Activation('relu', name='relu_2_2')(norm_c2_2)
    #max_c_3=MaxPool2D((2,2), strides=(2,2), padding='same') (relu_c2_2) 
   #############################################################3
    ############# Third CNN Residual Module
    
    conv_d = Conv2D(64, (3,3), padding='same', activation='relu', name='conv_d')(input_)
      
    
    conv_d_1 = Conv2D(128, (3,3), padding='same', activation='relu', name='conv_d_1')(conv_d)
    conv_d_2 = Conv2D(128, (2,2), padding='same', activation='relu', name='conv_d_2')(conv_d_1)
    conv_d_3 = Conv2D(64, (2,2), padding='same', activation='relu', name='conv_d_3')(conv_d_2)



# concatenate filters, assumes filters/channels last
    layer_out_d = Add()([conv_d_3, conv_d])
    layer_out_d = Activation('relu')(layer_out_d)
    max_d_3=MaxPool2D((2,2), strides=(4,4), padding='same') (layer_out_d) 
    
    
    ####################
    ##########################################concate first  and Second  CNN to Third CNN
    
    concate_level_2 = concatenate([relu_c2_2, max_d_3])
    
    conv_e_1 = Conv2D(64, kernel_size=(3, 3), padding='same', name='conv_5_1')(concate_level_2)
    norm_e_1 = BatchNormalization(name='norm_5_1')(conv_e_1)
    relu_e_1 = Activation('relu', name='relu_5_1')(norm_e_1)
 
    pool_e_1 = AveragePooling2D(pool_size=(2, 2), strides=1, padding='same', name='avg_pool_5_1')(relu_e_1)
    
    
    
    flatten_1=Flatten()(pool_e_1)

 
    dense_layer1 = Dense(units=100, activation='relu')(flatten_1)
    dense_layer1 = Dropout(0.4)(dense_layer1)
    dense_layer2 = Dense(units=50, activation='relu')(dense_layer1)
    dense_layer2 = Dropout(0.4)(dense_layer2)
    output_layer = Dense(units=output_units, activation='softmax')(dense_layer2)
 
    model = Model(inputs=input_, outputs=output_layer)
    model.summary()
    #plot_model(model, to_file='D:/Madianpari/WetlandsData/SpectralNET/data/wavelet_cnn_0.5.png')
    plot_model(model, to_file='wavelet_Flevoland.png', show_shapes=True, show_layer_names=True)
    #tf.keras.utils.plot_model(model, to_file='D:/Madianpari/WetlandsData/SpectralNET/data/model.png', show_shapes=False, show_dtype=False,show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96)


    return model

In [ ]:
model = get_wavelet_cnn_model()

In [ ]:
weight_decay = 0.0001
batch_size = 256
dropout_rate = 0.4
learning_rate = 0.001

In [ ]:

optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

checkpoint_filepath = "SpectralNet_Flev"
checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

history = model.fit(
        x=Xtrain,
        y=ytrain,
        batch_size=batch_size,
        epochs=100,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )


In [ ]:
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train and Validation Losses Over Epochs", fontsize=14)
plt.legend()
plt.grid()


plt.savefig('Flev.tiff',facecolor='w', dpi=500)
plt.show()


In [ ]:
Xtest = Xtest.reshape(-1, 12, 12, 12)


Xtest.shape

In [ ]:
y_pred_test = model.evaluate(Xtest, ytest)


In [ ]:
y_pred_test = model.predict(Xtest)
y_pred_test = np.argmax(y_pred_test, axis=1)
y_pred_test.shape

In [ ]:
ca = np.sum(y_pred_test == ytest) / ytest.shape[0]

print("Classification accuracy: %.5f" % ca)

In [ ]:
classification = classification_report( ytest, y_pred_test)
print(classification)